In [1]:
from funs.download import *
from funs.dataset import *
from funs.utils import *
from funs.databuilder import *
from funs.model import STgramMFN
from funs.trainer import Trainer
from funs.loss import CrossEntropyLoss

import os
import torch

from torch.utils.data import DataLoader

config = load_yaml('./config.yaml')

set_seed(config.seed)

train_dirs = [os.path.join(config.data_dir, config.snr, train_dir) for train_dir in config.train_dirs]
val_dirs = [os.path.join(config.data_dir, config.snr, val_dir) for val_dir in config.val_dirs]
test_dirs = [os.path.join(config.data_dir, config.snr, test_dir) for test_dir in config.test_dirs]

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

df = {}
df = download_mimii(config.data_dir, config.snr, config.sample_rate)

./data/0_dB/fan.zip already exist.
./data/0_dB/fan.zip already extracted.
./data/0_dB/pump.zip already exist.
./data/0_dB/pump.zip already extracted.
./data/0_dB/slider.zip already exist.
./data/0_dB/slider.zip already extracted.
./data/0_dB/valve.zip already exist.
./data/0_dB/valve.zip already extracted.


Processing fan - id_00 - abnormal: 407it [00:02, 151.80it/s]
Processing fan - id_00 - normal: 1011it [00:05, 176.95it/s]
Processing fan - id_02 - abnormal: 359it [00:01, 179.98it/s]
Processing fan - id_02 - normal: 1016it [00:05, 178.62it/s]
Processing fan - id_04 - abnormal: 348it [00:01, 177.75it/s]
Processing fan - id_04 - normal: 1033it [00:05, 176.93it/s]
Processing fan - id_06 - abnormal: 361it [00:02, 175.08it/s]
Processing fan - id_06 - normal: 1015it [00:05, 176.34it/s]
Processing pump - id_00 - abnormal: 143it [00:00, 178.33it/s]
Processing pump - id_00 - normal: 1006it [00:05, 175.48it/s]
Processing pump - id_02 - abnormal: 111it [00:00, 180.71it/s]
Processing pump - id_02 - normal: 1005it [00:05, 177.82it/s]
Processing pump - id_04 - abnormal: 100it [00:00, 181.35it/s]
Processing pump - id_04 - normal: 702it [00:03, 177.67it/s]
Processing pump - id_06 - abnormal: 102it [00:00, 181.63it/s]
Processing pump - id_06 - normal: 1036it [00:05, 178.15it/s]
Processing slider - id_00

In [12]:
id_list = df['model_id'].unique()
machine_list = df['machine_type'].unique()

In [3]:
dic = {}

label = 0
for machine in machine_list:
    for id in id_list:
        dic[f'{machine}-{id}'] =  label
        label += 1

dic

{'fan-id_00': 0,
 'fan-id_02': 1,
 'fan-id_04': 2,
 'fan-id_06': 3,
 'pump-id_00': 4,
 'pump-id_02': 5,
 'pump-id_04': 6,
 'pump-id_06': 7,
 'slider-id_00': 8,
 'slider-id_02': 9,
 'slider-id_04': 10,
 'slider-id_06': 11,
 'valve-id_00': 12,
 'valve-id_02': 13,
 'valve-id_04': 14,
 'valve-id_06': 15}

In [4]:
label_dic = make_meta2label(df)

train_df, val_df, test_df = split_df(df, config.val_model_id, config.test_model_id, config.seed)

In [25]:
train_data, train_label = build_from_dataframe(train_df)
# val_data, val_label = build_from_dataframe(val_df)
# test_data, test_label = build_from_dataframe(test_df)

In [7]:
# np.array(train_df['data'])
# np.array(train_df["machine_type"]+"-"+train_df["model_id"])


array(['fan-id_00', 'fan-id_00', 'fan-id_00', ..., 'valve-id_06',
       'valve-id_06', 'valve-id_06'], dtype=object)

In [8]:
# np.array(train_df['data'])

array([array([-0.01203156, -0.01239395, -0.01213455, ..., -0.00082016,
              -0.00095749, -0.00053024], dtype=float32)               ,
       array([-0.00137329, -0.00251007, -0.00389862, ...,  0.00708008,
               0.0063324 ,  0.00560379], dtype=float32)               ,
       array([-0.001297  , -0.00344086, -0.00466156, ...,  0.00073624,
               0.00021362,  0.00140762], dtype=float32)               ,
       ...,
       array([ 0.00022125,  0.00025558, -0.00016022, ..., -0.0045433 ,
              -0.00326157, -0.00211334], dtype=float32)               ,
       array([-0.00440216, -0.00170517,  0.00180817, ..., -0.00444412,
              -0.00265121, -0.00074005], dtype=float32)               ,
       array([-0.00255585, -0.00315094, -0.0022316 , ...,  0.00070572,
              -0.00017166, -0.00183487], dtype=float32)               ],
      dtype=object)

In [24]:
train_data, train_label = build_from_dataframe(train_df, dic)
# train_df = MIMIIDataset(train_data, train_label, sample_rate)

TypeError: build_from_dataframe() takes 1 positional argument but 2 were given